In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import requests as req
from bs4 import BeautifulSoup
import re
import datetime
from datetime import date
from os import listdir
import glob

In [ ]:
lista_keywords = ['Negro de mierda','Mantero','Extranjeros ilegales','panchito','panchitos','maricón','Sudaca','espaldas mojadas','indio','trasvesti','inmigrantes ilegales','Moro','Moros','Travelo','Paki','Bujarra','sidoso','negrata','Bollera','mariposon','desviado','mariquita','sarasa','machorra','tortillera','comebollos','machurrona','Africano','Latino','virus chino','Homosexual','machupichu','Tiraflechas','afeminado','negrata','indígenas','Sin papeles']

In [ ]:
def url_extractor(lista):
    options = webdriver.ChromeOptions()
    options.add_argument('disable-infobars')
    for keyword in lista:
        #Apertura de página con selenium
        listing_links = []
        link = 'https://eltorotv.com/?s='+ keyword
        
        print(keyword)
        
        driver = webdriver.Chrome(executable_path="./drivers/chromedriver")
        driver.get(link)
        time.sleep(1)
        #Código para evitar que entren noticias no relacionadas.
        resultados = driver.find_elements_by_xpath('//*[@id="main"]/section/header/h1')
    
        status = 'se han obtenido resultados'
        for i in resultados:
            if i.text == 'No hay resultados para: '+ keyword:
                status = 'sin resultados'
            
           
        #Extracción de links con selenium.
        try:
            for i in link:
                links = driver.find_elements_by_css_selector('.post-link')
                for link in links:
                    listing_links.append(str(link.get_attribute('href')))
                time.sleep(1)
                actions = ActionChains(driver) 
                #Scroll para qevitar banners
                for _ in range(6):
                    actions.send_keys(Keys.PAGE_DOWN).perform()
                time.sleep(1)
                driver.find_element_by_xpath("//*[text()='Entradas anteriores']").click()
                driver.close()
        except:
           
            #creación de dataframe con los datos generados
            listing_links = list(dict.fromkeys(listing_links))
            df = pd.DataFrame(listing_links, columns=['link'])
            df['keyword'] = keyword
            df['Status'] = status

            #guardado de archivos, con el nombre de su keyword
            nombre_archivo = keyword+ '_' + status + '.csv'
            path = './news_urls/el_gato_al_agua_urls'
            archivo_de_salida = os.path.join(path,nombre_archivo)
            print(df)

            df.to_csv(archivo_de_salida, index=False)

In [ ]:
def concatenator():
    path ='./news_urls/el_gato_al_agua_urls'
    e = glob.glob(path + "/*.csv")
    dfs = []
    for i in e:
        dfs.append(pd.read_csv(i))
        
    df_elgato_total = pd.concat(dfs, axis=0)
    df_elgato = df_elgato_total[df_elgato_total['Status'] != 'sin resultados']
    
    return df_elgato

In [ ]:
def scrap_news_function(df_elgato):
    df_elgato = df_elgato
    options = webdriver.ChromeOptions()
    options.add_argument('disable-infobars')
    links = df_elgato['link'].tolist()
    print(links)
    df_list = []
    for e in links:
        link = e
        driver = webdriver.Chrome(executable_path="../DFGM/drivers/chromedriver")
        driver.get(link)
        time.sleep(1)
        
        #Extracción del texto
        texto = driver.find_element_by_xpath('//*[@id="primary"]/div').text
        texto = texto.split('@')[0]
        texto = texto.replace('\n', ' ')
            
        #Extracción del titular
        titular = driver.find_element_by_css_selector('.entry-title').text
        
        date = driver.find_element_by_css_selector('.entry-date').text
        
        autor = driver.find_element_by_css_selector('.entry-author').text
        
        elgato_dict = {'newspaper': 'eltorotv.com', 'link': link,'title': titular, 'author': autor, 'date': date, 'text': texto}
        
        df_elgato_ = pd.DataFrame(elgato_dict, index=[0])
        
        df_elgato_['text'] = df_elgato_['text'].str.replace('\w.*\d+.', '')  
        
        driver.close()
        
        df_list.append(df_elgato_)
        
    df_final_gato = pd.concat(df_list)
    
    df_ = df_elgato.merge(df_final_gato, how='left')
        
    df_ = df_[['keyword', 'newspaper','link', 'title', 'author', 'date', 'text']]
    
    
    return df_

In [ ]:
def save_function(df_to_save):
    today = str(date.today())
    nombre_archivo = 'noticias_elgato_' + today + '.csv'
    path = '../DFGM/DATA_news'
    archivo_de_salida = os.path.join(path,nombre_archivo)
    df_to_save.to_csv(archivo_de_salida, index=False)

In [ ]:
def el_gato_pipeline(lista_keywords):
    
    starttime = datetime.datetime.now()
    url_extractor(lista_keywords)
    df_elgato = concatenator()
    
    df_to_save = scrap_news_function(df_elgato)
    save_function(df_to_save)

    endtime = datetime.datetime.now()
    duration = endtime - starttime
    
    print(duration)
    return df_to_save

In [ ]:
df = el_gato_pipeline(lista_keywords)

In [22]:
def url_extractor(lista):
    options = webdriver.ChromeOptions()
    options.add_argument('disable-infobars')
    for keyword in lista:
        #Apertura de página con selenium
        listing_links = []
        page = list(range(10001))
        
        for n_page in page[1:10001:50]:
            link = 'https://ariadna.elmundo.es/buscador/archivo.html?q='+ keyword + '&t=1&i=' + str(n_page) + '&n=50&fd=0&td=0&w=70&s=1&no_acd=1'

            print(keyword)

            driver = webdriver.Chrome(executable_path="../drivers/chromedriver")
            driver.get(link)
            time.sleep(1)

            #Extracción de links con selenium.
            for i in link:
                    links = driver.find_elements_by_css_selector('html body#buscador div#contenedor div#contenido div#contenido_principal div.bloque.bloque_noticias div.wrap div.col.col_01 ul.lista_resultados li h3 a')
                    for link in links:
                        listing_links.append(str(link.get_attribute('href')))
            
            driver.close()


        
        
            
        return listing_links
           

In [23]:
lst = ['covid']

In [24]:
x = url_extractor(lst)

covid
covid


KeyboardInterrupt: 

In [10]:
x

['https://www.elmundo.es/madrid/2020/05/07/5eb3e11afdddff59378b45f9.html',
 'https://www.elmundo.es/madrid/2020/05/07/5eb3e39afc6c8345038b459b.html',
 'https://www.elmundo.es/ciencia-y-salud/salud/2020/05/07/5eb397d5fdddffd93d8b45f7.html',
 'https://www.elmundo.es/television/medios/2020/05/07/5eb3cb29fdddff563f8b45e6.html',
 'https://www.elmundo.es/espana/2020/05/07/5eb3cfe721efa00e518b4612.html',
 'https://www.elmundo.es/ciencia-y-salud/salud/2020/05/07/5eb3cffb21efa0dc648b4623.html',
 'https://www.elmundo.es/madrid/2020/05/07/5eb3d62cfdddff4d988b4606.html',
 'https://www.elmundo.es/economia/macroeconomia/2020/05/07/5eb30c02fc6c83a00c8b459d.html',
 'https://www.elmundo.es/f5/mira/2020/05/07/5eb3b19afc6c83197f8b45a0.html',
 'https://www.elmundo.es/espana/2020/05/07/5eb2ea01fc6c8314388b469a.html',
 'https://www.elmundo.es/espana/2020/04/29/5ea8aab4fdddff4a238b45d9.html',
 'https://www.elmundo.es/espana/2020/05/03/5eaaf7affdddff46138b4591.html',
 'https://www.elmundo.es/espana/2020/04/30

In [2]:
link = 'https://ariadna.elmundo.es/buscador/archivo.html?q=covid&t=1&n=50&w=70&s=1'


driver = webdriver.Chrome(executable_path="../drivers/chromedriver")
driver.get(link)
time.sleep(1)

#Extracción de links con selenium.
listing_links = []
for i in link:
    links = driver.find_elements_by_css_selector('html body#buscador div#contenedor div#contenido div#contenido_principal div.bloque.bloque_noticias div.wrap div.col.col_01 ul.lista_resultados li h3 a')
    for link in links:
        listing_links.append(str(link.get_attribute('href')))

In [3]:
listing_links

['https://www.elmundo.es/ciencia-y-salud/salud/2020/05/07/5eb397d5fdddffd93d8b45f7.html',
 'https://www.elmundo.es/television/medios/2020/05/07/5eb3cb29fdddff563f8b45e6.html',
 'https://www.elmundo.es/madrid/2020/05/07/5eb3e11afdddff59378b45f9.html',
 'https://www.elmundo.es/espana/2020/05/07/5eb3cfe721efa00e518b4612.html',
 'https://www.elmundo.es/ciencia-y-salud/salud/2020/05/07/5eb3cffb21efa0dc648b4623.html',
 'https://www.elmundo.es/madrid/2020/05/07/5eb3d62cfdddff4d988b4606.html',
 'https://www.elmundo.es/economia/macroeconomia/2020/05/07/5eb30c02fc6c83a00c8b459d.html',
 'https://www.elmundo.es/f5/mira/2020/05/07/5eb3b19afc6c83197f8b45a0.html',
 'https://www.elmundo.es/espana/2020/05/07/5eb2ea01fc6c8314388b469a.html',
 'https://www.elmundo.es/espana/2020/04/29/5ea8aab4fdddff4a238b45d9.html',
 'https://www.elmundo.es/espana/2020/05/03/5eaaf7affdddff46138b4591.html',
 'https://www.elmundo.es/espana/2020/04/30/5eaaf011fdddff708e8b45b3.html',
 'https://www.elmundo.es/espana/2020/05/07

NameError: name 'x' is not defined

In [14]:
page = list(range(10001))

In [ ]:
x = page[1:10001:50]

In [20]:
x

[1,
 51,
 101,
 151,
 201,
 251,
 301,
 351,
 401,
 451,
 501,
 551,
 601,
 651,
 701,
 751,
 801,
 851,
 901,
 951,
 1001,
 1051,
 1101,
 1151,
 1201,
 1251,
 1301,
 1351,
 1401,
 1451,
 1501,
 1551,
 1601,
 1651,
 1701,
 1751,
 1801,
 1851,
 1901,
 1951,
 2001,
 2051,
 2101,
 2151,
 2201,
 2251,
 2301,
 2351,
 2401,
 2451,
 2501,
 2551,
 2601,
 2651,
 2701,
 2751,
 2801,
 2851,
 2901,
 2951,
 3001,
 3051,
 3101,
 3151,
 3201,
 3251,
 3301,
 3351,
 3401,
 3451,
 3501,
 3551,
 3601,
 3651,
 3701,
 3751,
 3801,
 3851,
 3901,
 3951,
 4001,
 4051,
 4101,
 4151,
 4201,
 4251,
 4301,
 4351,
 4401,
 4451,
 4501,
 4551,
 4601,
 4651,
 4701,
 4751,
 4801,
 4851,
 4901,
 4951,
 5001,
 5051,
 5101,
 5151,
 5201,
 5251,
 5301,
 5351,
 5401,
 5451,
 5501,
 5551,
 5601,
 5651,
 5701,
 5751,
 5801,
 5851,
 5901,
 5951,
 6001,
 6051,
 6101,
 6151,
 6201,
 6251,
 6301,
 6351,
 6401,
 6451,
 6501,
 6551,
 6601,
 6651,
 6701,
 6751,
 6801,
 6851,
 6901,
 6951,
 7001,
 7051,
 7101,
 7151,
 7201,
 7251,
 

In [1]:
'hola'

'hola'